## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-17-10-52-09 +0000,nypost,Pentagon blasts Netflix’s new gay military ser...,https://nypost.com/2025/10/17/us-news/pentagon...
1,2025-10-17-10-51-10 +0000,bbc,Minister to meet officials over 'unacceptable'...,https://www.bbc.com/news/articles/c205gnz5p8xo...
2,2025-10-17-10-49-39 +0000,nyt,Explosion Destroys Vehicles Outside Home of It...,https://www.nytimes.com/2025/10/17/world/europ...
3,2025-10-17-10-49-00 +0000,wsj,Financial Services Roundup: Market Talk,https://www.wsj.com/articles/financial-service...
4,2025-10-17-10-46-31 +0000,nyt,U.S. Says Louisiana Man Joined Hamas-Led 2023 ...,https://www.nytimes.com/2025/10/17/us/fbi-hama...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
71,trump,64
3,new,24
443,will,16
483,china,14
445,gaza,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
211,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...,140
270,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...,129
163,2025-10-16-23-01-00 +0000,wsj,Trump Says He Will Meet With Putin in Budapest...,https://www.wsj.com/world/russia/trump-and-put...,127
322,2025-10-16-14-49-58 +0000,nypost,Trump says CBS under owner David Ellison will ...,https://nypost.com/2025/10/16/media/trump-says...,127
303,2025-10-16-15-47-55 +0000,cbc,"Trump's Gaza deal may be 'historic,' but falls...",https://www.cbc.ca/news/world/gaza-trump-peace...,119


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
211,140,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...
22,67,2025-10-17-10-00-00 +0000,wapo,"The average new-car price hit a record $50,000...",https://www.washingtonpost.com/business/2025/1...
99,67,2025-10-17-03-46-08 +0000,nypost,"Zohran Mamdani dodges attacks, straight answer...",https://nypost.com/2025/10/16/us-news/mamdani-...
297,47,2025-10-16-16-00-00 +0000,wsj,An extended government shutdown raises the pro...,https://www.wsj.com/economy/central-banking/fe...
171,46,2025-10-16-22-31-31 +0000,startribune,"Prison for woman who admitted she was driving,...",https://www.startribune.com/prison-for-woman-a...
333,45,2025-10-16-14-00-14 +0000,nypost,"Chaos erupts after Kenyan forces fire in air, ...",https://nypost.com/2025/10/16/world-news/kenya...
223,45,2025-10-16-20-21-13 +0000,nypost,Thune says ‘time’s right’ for Russia sanctions...,https://nypost.com/2025/10/16/us-news/thune-sa...
27,44,2025-10-17-09-49-56 +0000,bbc,Zelensky to make case for missiles at White Ho...,https://www.bbc.com/news/articles/crmxz37nv3zo...
183,42,2025-10-16-21-54-57 +0000,cbc,"John Bolton, Trump's ex-national security advi...",https://www.cbc.ca/news/world/john-bolton-char...
360,41,2025-10-16-12-11-11 +0000,wapo,Hamas took 251 hostages from Israel into Gaza....,https://www.washingtonpost.com/world/interacti...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
